In [6]:
from publisher.logging.logger import logger


---
# Application


In [8]:
import requests

from publisher.models.news_response import NewsResponse
from publisher.models.news import News


In [2]:
url = "http://127.0.0.1:8000/v1"

endpoints = {
    "news": "/news",
}


In [4]:
news_response = NewsResponse(**requests.get("http://127.0.0.1:8000/news/?sources=tagesschau&keywords=hive").json())


In [5]:
news_response


NewsResponse(metadata=ResponseMetadata(pages=1, page=1, timestamp=datetime.datetime(2023, 1, 26, 21, 13, 56, 317387)), response=[News(source='tagesschau', title='BW-Strafverfolgungsbehörden beteiligt an Schlag gegen Cyber-Kriminelle', sub_title='Internationale Strafverfolgungsbehörden und US-Behörden sind gegen die Ransomware-Gruppe Hive vorgegangen.', tags=['baden-württemberg'], text=None, url='https://www.tagesschau.de/ardimport/regional/badenwuerttemberg/swr-bw-story-47143.html', timestamp=datetime.datetime(2023, 1, 26, 20, 43, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))), News(source='tagesschau', title='Hackernetzwerk "Hive" zerschlagen', sub_title='Es wird für weltweit mehr als 1500 schwere Cyberangriffe auf Unternehmen verantwortlich gemacht.', tags=['cyberattacke', 'hacker', 'hackernetzwerk', 'cyberangriff', '"hive"', 'cyber'], text=None, url='https://www.tagesschau.de/wirtschaft/unternehmen/zerschlagung-hackernetzwerk-hive-101.html', timestamp=datetime.datetime

In [16]:
def get_news(
    keywords: list[str],
    sources: list[str] = ["tagesschau"],
    url: str = "http://127.0.0.1:8000",
    endpoint: str = "/news",
) -> list[News]:

    joined_sources = "&".join((f"sources={source}" for source in sources))
    joined_keywords = "&".join((f"keywords={keyword}" for keyword in keywords))

    query_parameters = "&".join((joined_sources, joined_keywords))

    request_url = f"{url}{endpoint}/?{query_parameters}"
    print(request_url)

    return NewsResponse(**requests.get(request_url).json()).response


---
# ntfy


In [10]:
ntfy_url = "https://ntfy.sh"
topic = "/news-ticker"


def post_news(
    news: News,
    topic: str=topic,
    url: str=ntfy_url,
) -> None:
    logger.info("functionCall:  post_news()")

    request_url = f"{url}{topic}"
    headers={
        "Title": news.title,
        "Click": news.url,
        # "Priority": "urgent",
        # "Tags": "warning,skull"
    }
    data = news.sub_title

    logger.debug(f"request_url:  {request_url}")
    logger.debug(f"headers:  {headers}")
    logger.debug(f"data:  {data}")

    requests.post(
        url=request_url,
        headers=headers,
        data=data
    )


In [19]:
from time import sleep

keywords = [
    "bnd",
    "kampfpanzer",
    "hacker",
]

news_response = get_news(keywords)

for news in news_response:
    sleep(2)
    post_news(news)


http://127.0.0.1:8000/news/?sources=tagesschau&keywords=bnd&keywords=kampfpanzer&keywords=hacker
